In [125]:
import numpy as np
import pandas as pd
import matplotlib as plot
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
import pyecharts
import time
import torch

In [126]:
df_data = pd.read_excel(r'./宁夏省 .xlsx')
data = df_data.iloc[0:]
data = pd.DataFrame(data)
print(data.info())
print(max(data['确诊人数']))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 5 columns):
日期         236 non-null object
确诊人数       236 non-null int64
迁入指数       236 non-null float64
迁出指数       236 non-null float64
迁入迁出总指数    236 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 9.3+ KB
None
6


In [127]:
# 将日期的数据类型改为datatime
data['日期'] = pd.to_datetime(data['日期'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 5 columns):
日期         236 non-null datetime64[ns]
确诊人数       236 non-null int64
迁入指数       236 non-null float64
迁出指数       236 non-null float64
迁入迁出总指数    236 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 9.3 KB


In [164]:
# 对于宁夏省每日新增确诊病例趋势可视化
line = (Line(init_opts=opts.InitOpts(theme='blue', width='1000px'))
        .add_xaxis([day.strftime('%Y-%m-%d') for day in data['日期']][0:])
        .add_yaxis("宁夏省当日新增人数", data['确诊人数'],is_smooth=True,
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.3, color='yellow'))
        .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
        .set_global_opts(
            datazoom_opts=[opts.DataZoomOpts(range_start=0, range_end=1)],
            title_opts=opts.TitleOpts(title="宁夏省每日新增确证病例趋势图",
                                     subtitle="更新时间：{}".format(data['日期'].max())),
            xaxis_opts=opts.AxisOpts(
                   splitline_opts=opts.SplitLineOpts(is_show=False)),
            legend_opts=opts.LegendOpts(is_show=True),
       ))

line.render_notebook()

In [129]:
# 宁夏省的迁出人数和确诊人数的趋势图
line = (Line(init_opts=opts.InitOpts(width='900px'))
        .add_xaxis([day.strftime('%Y-%m-%d') for day in data['日期']][0:])
        .add_yaxis("宁夏省当日新增人数", data['确诊人数'],is_smooth=True,
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.1,color='yellow'))
        .add_yaxis("宁夏省当日迁出指数", round(data['迁出指数'], 2),is_smooth=True,
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.1,color='blue'))
        .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
        .set_global_opts(
            datazoom_opts=[opts.DataZoomOpts(range_start=0, range_end=1)],
            title_opts=opts.TitleOpts(title="宁夏省每日新增确诊和迁出指数趋势图",
                                     subtitle="更新时间：{}".format(data['日期'].max())),
            xaxis_opts=opts.AxisOpts(
                   splitline_opts=opts.SplitLineOpts(is_show=False)),
            legend_opts=opts.LegendOpts(is_show=True),
       ))

line.render_notebook()

In [132]:
def xiangcheng(x,y):
    return x*y
def suanmo(x,y):
    return np.sqrt(x**2)+np.sqrt(y**2)
def smiliar(x,y):
    return xiangcheng(x,y)/suanmo(x,y)


people_of_NCP_list = list(data['确诊人数'])
people_of_NCP_tensor = torch.Tensor(people_of_NCP_list)
data_out_list = list(data['迁出指数'])
data_out_tensor = torch.Tensor(data_out_list)
result = []
for i in range(len(data_out_list)):
    sum = smiliar(people_of_NCP_tensor[i],data_out_tensor[i])
    result.append(round(sum.item(), 2))
    
# 将数据保存到excel中
df = pd.DataFrame(result, columns=['相关指数'])
df.to_excel("./宁夏省相关指数 .xlsx",index = False)
# 读取数据
index_smiliar = pd.read_excel(r'./宁夏省相关指数 .xlsx')
data_index = index_smiliar['相关指数']

In [166]:
#画出确诊人数和迁出指数相关系数趋势图
line = (Line(init_opts=opts.InitOpts(width='900px'))
        .add_xaxis([day.strftime('%Y-%m-%d') for day in data['日期']][0:])
        .add_yaxis("确诊人数和迁出指数相关系数", data_index,is_smooth=True,
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.1,color='red'))
        .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
        .set_global_opts(
            datazoom_opts=[opts.DataZoomOpts(range_start=0, range_end=1)],
            title_opts=opts.TitleOpts(title="确诊人数和迁出指数相关系数趋势图",
                                     subtitle="更新时间：{}".format(data['日期'].max())),
            xaxis_opts=opts.AxisOpts(
                   splitline_opts=opts.SplitLineOpts(is_show=False)),
            legend_opts=opts.LegendOpts(is_show=True),
       ))

line.render_notebook()

In [140]:
data_2 = index_smiliar.iloc[188:]
line = (Line(init_opts=opts.InitOpts(width='900px'))
        .add_xaxis([day.strftime('%Y-%m-%d') for day in data['日期']][188:])
        .add_yaxis("确诊人数和迁出指数相关系数", data_2['相关指数'],is_smooth=True,
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.1,color='greed'))
        .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
        .set_global_opts(
            datazoom_opts=[opts.DataZoomOpts(range_start=0, range_end=30)],
            title_opts=opts.TitleOpts(title="新冠发生前后14天确诊人数和迁出指数相关系数趋势图",
                                     subtitle="更新时间：{}".format(data['日期'].max())),
            xaxis_opts=opts.AxisOpts(
                   splitline_opts=opts.SplitLineOpts(is_show=False)),
            legend_opts=opts.LegendOpts(is_show=True),
       ))

line.render_notebook()

In [162]:
data1 = pd.read_excel(r'./宁夏省 .xlsx',sheet_name='银川')
data2 = pd.read_excel(r'./宁夏省 .xlsx',sheet_name='固原')
data3 = pd.read_excel(r'./宁夏省 .xlsx',sheet_name='吴忠')
data4 = pd.read_excel(r'./宁夏省 .xlsx',sheet_name='中卫')
data5 = pd.read_excel(r'./宁夏省 .xlsx',sheet_name='石嘴山')
Data_1 = data1.iloc[202:]
Data_2 = data2.iloc[202:]
Data_3 = data3.iloc[202:]
Data_4 = data4.iloc[202:]
Data_5 = data5.iloc[202:]
line = (Line(init_opts=opts.InitOpts(width='900px'))
        .add_xaxis([day.strftime('%Y-%m-%d') for day in data['日期']][202:])
        .add_yaxis("新冠发生后银川迁出指数", round(Data_1['迁出指数'], 2),is_smooth=True,
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.0,color='greed'))
        .add_yaxis("新冠发生后固原迁出指数", round(Data_2['迁出指数'], 2),is_smooth=True,
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.0,color='greed'))
        .add_yaxis("新冠发生后吴忠迁出指数", round(Data_3['迁出指数'], 2),is_smooth=True,
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.0,color='greed'))
        .add_yaxis("新冠发生后中卫迁出指数", round(Data_4['迁出指数'], 2),is_smooth=True,
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.0,color='greed'))
        .add_yaxis("新冠发生后石嘴山迁出指数", round(Data_5['迁出指数'], 2),is_smooth=True,
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.0,color='greed'))
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            datazoom_opts=[opts.DataZoomOpts(range_start=0, range_end=1)],
            title_opts=opts.TitleOpts(title="新冠发生后各市每日迁出指数趋势图",
                                     subtitle="更新时间：{}".format(data['日期'].max())),
            xaxis_opts=opts.AxisOpts(
                   splitline_opts=opts.SplitLineOpts(is_show=False)),
            legend_opts=opts.LegendOpts(
                    pos_right="30%",
                    pos_top='5%',
                    pos_bottom="10%",
                    orient='vertical',
                    item_width=25,
            )))

line.render_notebook()

In [165]:
data1 = pd.read_excel(r'./宁夏省 .xlsx',sheet_name='银川')
data2 = pd.read_excel(r'./宁夏省 .xlsx',sheet_name='固原')
data3 = pd.read_excel(r'./宁夏省 .xlsx',sheet_name='吴忠')
data4 = pd.read_excel(r'./宁夏省 .xlsx',sheet_name='中卫')
data5 = pd.read_excel(r'./宁夏省 .xlsx',sheet_name='石嘴山')
Data_1 = data1.iloc[202:]
Data_2 = data2.iloc[202:]
Data_3 = data3.iloc[202:]
Data_4 = data4.iloc[202:]
Data_5 = data5.iloc[202:]
line = (Line(init_opts=opts.InitOpts(width='900px'))
        .add_xaxis([day.strftime('%Y-%m-%d') for day in data['日期']][202:])
        .add_yaxis("新冠发生后银川确诊人数", round(Data_1['确诊人数'], 2),is_smooth=True,
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.0,color='greed'))
        .add_yaxis("新冠发生后固原确诊人数", round(Data_2['确诊人数'], 2),is_smooth=True,
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.0,color='greed'))
        .add_yaxis("新冠发生后吴忠确诊人数", round(Data_3['确诊人数'], 2),is_smooth=True,
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.0,color='greed'))
        .add_yaxis("新冠发生后中卫确诊人数", round(Data_4['确诊人数'], 2),is_smooth=True,
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.0,color='greed'))
        .add_yaxis("新冠发生后石嘴山确诊人数", round(Data_5['确诊人数'], 2),is_smooth=True,
                   areastyle_opts=opts.AreaStyleOpts(opacity=0.0,color='greed'))
        .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
        .set_global_opts(
            datazoom_opts=[opts.DataZoomOpts(range_start=0, range_end=1)],
            title_opts=opts.TitleOpts(title="新冠发生后各市每日确诊人数趋势图",
                                     subtitle="更新时间：{}".format(data['日期'].max())),
            xaxis_opts=opts.AxisOpts(
                   splitline_opts=opts.SplitLineOpts(is_show=False)),
            legend_opts=opts.LegendOpts(
                    pos_right="30%",
                    pos_top='5%',
                    pos_bottom="10%",
                    orient='vertical',
                    item_width=55,
            )))

line.render_notebook()